### Herramientas recomendadas
- **PyTorch**: Framework para el desarrollo de redes neuronales.
- **Weights and Biases**: Herramienta para monitoreo y visualización de métricas de entrenamiento.
- **Google Colab**: Si no cuentan con GPU local, pueden usar Google Colab para ejecutar el proyecto. [Guía completa para conectar Google Drive a Google Colab](https://medium.com/@DataScience-ProF/a-comprehensive-guide-to-connecting-google-drive-to-google-colab).
- **OpenCV**: Para aplicar filtros de imagen y otras transformaciones.

---

### Pasos a seguir
En síntesis, el trabajo incluye: los estudiantes deben realizar clasificadores CNN utilizando el dataset **Covid-19 Image Dataset**, el cual contiene imágenes de rayos X y está clasificado en tres clases: **Covid-19**, **Normal**, y **Neumonía**.

#### 4.1 Selección del modelo
1. Seleccionen dos modelos de convolución preexistentes como **ResNet**, **AlexNet**, **VGG16**, o cualquiera de los conversados en clase.
2. Descarguen el modelo preentrenado y realicen **fine-tuning** para adaptarlo al problema.
3. Justifiquen la elección de ambos modelos.

#### 4.2 Preprocesamiento de imágenes
Entrenen los modelos utilizando tres conjuntos de datos diferentes:
- **Conjunto 1**: Usar las imágenes tal como están (crudas/raw).
- **Conjunto 2**: Aplicar un filtro bilateral para suavizar la imagen y eliminar el ruido.
- **Conjunto 3**: Aplicar un filtro **Canny Edge** para detectar los bordes de las imágenes.

> **Consejo**: Es recomendable realizar el preprocesamiento de filtros antes del entrenamiento para hacer un uso eficiente del poder computacional.


## Descarga del Dataset de Covid

In [3]:
import kagglehub
import shutil


# Download the dataset (default download path will be in /root/.kagglehub)
path = kagglehub.dataset_download("pranavraikokte/covid19-image-dataset")

# Specify the destination path in Google Drive
save_path = "dataset"

# Move the downloaded dataset to the specified path in Google Drive
shutil.move(path, save_path)

print("Dataset successfully moved to:", save_path)


Dataset successfully moved to: dataset


## Preparando Pytorch

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import cv2
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class CovidDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []


# Define transformations
data_transforms = {
    'raw': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load pre-trained models
model_resnet = models.resnet18(pretrained=True).to(device)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 3).to(device)  # 3 output classes

model_vgg = models.vgg16(pretrained=True).to(device)
model_vgg.classifier[6] = nn.Linear(4096, 3).to(device) # 3 output classes

# Freeze the pre-trained layers
for param in model_resnet.parameters():
    param.requires_grad = False
for param in model_vgg.parameters():
    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 209MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

## Preparando el Dataset

In [6]:
class CovidDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = ['covid', 'normal', 'pneumonia']

        # Iterate through each class folder and append file paths and labels
        for idx, class_name in enumerate(self.class_names):
            class_dir = os.path.join(image_dir, class_name)
            for img_file in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_file))
                self.labels.append(idx)  # Assign label based on class index

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Read image using PIL
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


# Resnet

In [ ]:
# Paths
dataset_dir = "/content/drive/MyDrive/Covid19_Image_Dataset/train"  # Update with your actual paths
val_dir = "/content/drive/MyDrive/Covid19_Image_Dataset/test"

# Datasets
train_dataset = CovidDataset(image_dir=dataset_dir, transform=data_transforms['raw'])
val_dataset = CovidDataset(image_dir=val_dir, transform=data_transforms['raw'])
                                                                  
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


NameError: name 'CovidDataset' is not defined

## VGG